# multiprocessing test
## multiprocessing.Poolを使用する例
- with Pool(os.cpu_count()) as p:


In [ ]:
import os
from glob import glob
from multiprocessing.pool import ThreadPool as Pool
from func_test2 import thread_save
# from save_im import thread_save
print(os.cpu_count())

# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)

thread_save()


# multiprocessing + imap_unordered

In [ ]:
import os
from multiprocessing import Pool, cpu_count 
from tqdm import tqdm 
from glob import glob
from multiprocess_test import wrap_save_npy
top_results = [[0, 0] for _ in range(11)]

# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
p = Pool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

# processに渡す引数の作成
job_args = [(path, path.split('.')[1], num) for num,path in enumerate(path_list)]
print(job_args)
# imapによってprocessの順番は任意で処理される
for result in p.imap_unordered(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
    print(result)


# ParallelPool + uimap
- processのprintが反映される

In [ ]:
import os
from pathos.pools import ParallelPool
from tqdm import tqdm 
from glob import glob
from multiprocess_test import wrap_save_npy
top_results = [[0, 0] for _ in range(11)]

# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
p = ParallelPool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

# processに渡す引数の作成
job_args = [(path, path.split('.')[1], num) for num,path in enumerate(path_list)]
print(job_args)
# imapによってprocessの順番は任意で処理される
for result in p.uimap(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
    print(result)


# メインで関数化すると動かない
- pool + imap_unorderdはダメ
- ParallelPool+ uimapもダメ

In [ ]:
import os
from multiprocessing import Pool, cpu_count 
from glob import glob

import numpy as np
import cv2
from PIL import Image

save_base_path = 'gaussian_filtered_nparrays/'

# 結果保存用のリストを初期化
results = [[0, 0] for _ in range(11)]

# 引数をリストで受け取るラッパー
def wrap_save_npy(args):
    return _save_npy(*args)

#　実際のprocess処理
def _save_npy(path, type_name, num):
    global results
    #pngデータをopencvでロード
    im = cv2.imread(path)
    # わざとCPUバウンドの処理を入れる
    im_calc = Image.open(path).convert('L')
    im_calc = np.array(im)
    im_calc = im_calc.astype(np.float64)
    i = 6
    im_b = im_calc
    for _ in range(i):
        im_a = im_calc+np.var(im_calc)
        im_b = im_calc-np.var(im_b)
        im_calc = np.cov(im_b, im_a)
    x = np.argsort(im_calc.flatten())
    # 処理結果をglobalの結果ファイルに代入する
    results[num][0] = num
    results[num][1] = len(x)
    #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
    file_name = path.split('\\')[-1].split('.png')[0]
    save_path = save_base_path + file_name + '_' + type_name
    #全てのnpyファイルを同じフォルダに格納
    np.save(save_path, im)
    return save_path, results, num

top_results = [[0, 0] for _ in range(11)]

# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
p = Pool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

# processに渡す引数の作成
job_args = [(path, path.split('.')[1], num) for num,path in enumerate(path_list)]
print(job_args)
# imapによってprocessの順番は任意で処理される
for result in p.imap_unordered(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
    print(result)


In [ ]:
import os
from multiprocessing import  cpu_count 
from pathos.pools import ParallelPool
from glob import glob

import numpy as np
import cv2
from PIL import Image

save_base_path = 'gaussian_filtered_nparrays/'

# 結果保存用のリストを初期化
results = [[0, 0] for _ in range(11)]

# 引数をリストで受け取るラッパー
def wrap_save_npy(args):
    return _save_npy(*args)

#　実際のprocess処理
def _save_npy(path, type_name, num):
    global results
    #pngデータをopencvでロード
    im = cv2.imread(path)
    # わざとCPUバウンドの処理を入れる
    im_calc = Image.open(path).convert('L')
    im_calc = np.array(im)
    im_calc = im_calc.astype(np.float64)
    i = 6
    im_b = im_calc
    for _ in range(i):
        im_a = im_calc+np.var(im_calc)
        im_b = im_calc-np.var(im_b)
        im_calc = np.cov(im_b, im_a)
    x = np.argsort(im_calc.flatten())
    # 処理結果をglobalの結果ファイルに代入する
    results[num][0] = num
    results[num][1] = len(x)
    #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
    file_name = path.split('\\')[-1].split('.png')[0]
    save_path = save_base_path + file_name + '_' + type_name
    #全てのnpyファイルを同じフォルダに格納
    np.save(save_path, im)
    return save_path, results, num

top_results = [[0, 0] for _ in range(11)]

# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
p = ParallelPool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

# processに渡す引数の作成
job_args = [(path, path.split('.')[1], num) for num,path in enumerate(path_list)]
print(job_args)
# imapによってprocessの順番は任意で処理される
for result in p.uimap(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
    print(result)


# queueでの変数をJupyter上で試す

# Process + Queue＊２
- Queueを使うと引数にQueue以外が渡せない
- jupyter上では動作しない


In [ ]:
# FIXME:
from multiprocessing import Process, Queue, cpu_count
# import queue
import time
import numpy as np
start_t = time.time()
q = Queue()
output = Queue()
threads = []
max_worker = 10
#a_l = []
a_l = np.empty([max_worker])
print(a_l)
title = 'test'
def worker(q, output):
    global a_l, title
    while True:
        item = q.get()
        print(f'title = {title}')
        if item is None:
            output.put(item+title)
            break
        print(f'Working on {item}')
        time.sleep(0.1)
#        a_l.append(item)
        a_l[item] = item+title
        print(f'Finished {item}')
        
        q.task_done()

# Turn-on the worker thread.
for i in range(max_worker):
    t = Process(target=worker, args=(q, a_l, title), daemon=True)
    t.start()
    threads.append(t)
    
# for i in range(max_worker):
#     print(f'output = {output.get()}')
    
for item in range(max_worker):
    q.put(item)
    
for item in range(max_worker):
    q.put(None)
# Block until all tasks are done.
for t in threads:
    t.join()
    print(f'output = {output.get()}')
print(a_l)
print(time.time()-start_t)
print('All work completed')



# subprocessによるmultiprocessing

In [ ]:
import subprocess
from subprocess import PIPE

proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')

In [ ]:
import subprocess
from subprocess import PIPE

# proc = subprocess.run(['python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)

# venvでの動作にはvenv/Scripts以下のpythonを指定しないとimportモジュールが使用できなくなる可能性がある
# proc = subprocess.run(['venv/Scripts/python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
proc = subprocess.run(['venv/Scripts/python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)

print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')

In [ ]:
import subprocess
from subprocess import PIPE

# proc = subprocess.run(['python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)

# venvでの動作にはvenv/Scripts以下のpythonを指定しないとimportモジュールが使用できなくなる可能性がある
# proc = subprocess.run(['venv/Scripts/python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
proc = subprocess.run(['venv/Scripts/python', 'process_print_id_test.py'], stdout=PIPE, stderr=PIPE)

print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')
for line in out:
    print(line)

- 引数の受け渡し

In [ ]:
import subprocess
from subprocess import PIPE, Popen
import os

# subprocess.callは3.5以下で使用。通常はrun(推奨）かPopen
# proc = subprocess.run(['python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)

# venvでの動作にはvenv/Scripts以下のpythonを指定しないとimportモジュールが使用できなくなる可能性がある
# 仮想環境かかどうかのチェックが必要
# proc = subprocess.run(['venv/Scripts/python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
python_path = 'python'
if os.path.isdir('venv'):
    python_path = 'venv/Scripts/python'
proc = subprocess.run([python_path, 'process_print_id_get_arg_test.py', 'main line', 'bob' ], stdout=PIPE, stderr=PIPE)

print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')
for line in out:
    print(line)

- ndarrayの受け渡し

In [ ]:
import subprocess
from subprocess import PIPE, Popen
import os
import numpy as np

out_l = np.zeros((2,2))
print(out_l)
# input_array = 'out_l.npy'
input_array_path = os.path.abspath('out_l.npy')
print(f'input path = {input_array_path}:{type(input_array_path)}')
shape_x = 2
shape_y = 2

para_path = os.path.abspath('para.txt')
para = np.array([[input_array_path, shape_x, shape_y]], dtype=object)
# パラメータをファイルで渡す
np.savetxt(para_path, para, delimiter=',', fmt='%s')
# subprocess.callは3.5以下で使用。通常はrun(推奨）かPopen
# proc = subprocess.run(['python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)

# venvでの動作にはvenv/Scripts以下のpythonを指定しないとimportモジュールが使用できなくなる可能性がある
# 仮想環境かかどうかのチェックが必要
# 引数はintはダメでstrにキャストする必要がある。
# proc = subprocess.run(['venv/Scripts/python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
python_path = 'python'
if os.path.isdir('venv'):
    python_path = 'venv/Scripts/python'
print(f'python path = {python_path}:{type(python_path)}')
# proc = subprocess.run([python_path, 'process_print_id_get_arg_test.py', 'main line', 'bob' ], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run([python_path, 'process_share_ndarray_test.py', input_array_path, str(shape_x), str(shape_y) ], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run([python_path, 'process_share_ndarray_test.py', input_array_path, str(shape_x), str(shape_y) ], stdout=PIPE, stderr=PIPE)
proc = subprocess.run([python_path, 'process_share_ndarray_test.py', para_path ], stdout=PIPE, stderr=PIPE)

print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')
for line in out:
    print(line)
loaded_array = np.load(input_array_path)
print(f'result = {loaded_array}')

# multiprocessing+subprocess

In [1]:
import subprocess
from subprocess import PIPE, Popen
import os
import numpy as np

out_l = np.zeros((2,2))
print(out_l)
# input_array = 'out_l.npy'
input_array_path = os.path.abspath('out_l.npy')
print(f'input path = {input_array_path}:{type(input_array_path)}')
shape_x = 2
shape_y = 2

para_path = os.path.abspath('para.txt')
para = np.array([[input_array_path, shape_x, shape_y]], dtype=object)
# パラメータをファイルで渡す
np.savetxt(para_path, para, delimiter=',', fmt='%s')
# subprocess.callは3.5以下で使用。通常はrun(推奨）かPopen
# proc = subprocess.run(['python', 'multiprocess_module_test.py'], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run(['python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)

# venvでの動作にはvenv/Scripts以下のpythonを指定しないとimportモジュールが使用できなくなる可能性がある
# 仮想環境かかどうかのチェックが必要
# 引数はintはダメでstrにキャストする必要がある。
# proc = subprocess.run(['venv/Scripts/python', 'subprocess_test.py'], stdout=PIPE, stderr=PIPE)
python_path = 'python'
if os.path.isdir('venv'):
    python_path = 'venv/Scripts/python'
print(f'python path = {python_path}:{type(python_path)}')

n = 15
# proc = subprocess.run([python_path, 'process_print_id_get_arg_test.py', 'main line', 'bob' ], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run([python_path, 'process_share_ndarray_test.py', input_array_path, str(shape_x), str(shape_y) ], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run([python_path, 'process_share_ndarray_test.py', input_array_path, str(shape_x), str(shape_y) ], stdout=PIPE, stderr=PIPE)
# proc = subprocess.run([python_path, 'queue_async_share_ndarray_test.py', para_path, str(n) ], stdout=PIPE, stderr=PIPE)
# subprocess_cmd = [python_path, 'queue_async_share_ndarray_test.py', para_path, str(n) ]
subprocess_cmd = [python_path, 'queue_async_share_ndarray_test2.py', para_path, str(n) ]

proc = subprocess.run(subprocess_cmd, stdout=PIPE, stderr=PIPE)

print(proc.stdout.decode('utf-8').split('\r\n'))
print(proc.stderr.decode('utf-8').split('\n'))
out = proc.stdout.decode('utf-8').split('\r\n')
print(f'{out}:{type(out)}')
for line in out:
    print(line)
for i in range(n):
    splited_path = os.path.splitext(input_array_path)
    file_name = splited_path[0]+str(i)+splited_path[1]
    loaded_array = np.load(file_name)
    print(f'result = {loaded_array}')
print(f'top result = {np.load("test.npy")}')    

[[0. 0.]
 [0. 0.]]
input path = c:\Users\fujikawa\python_projects\ARIM_testcode\out_l.npy:<class 'str'>
python path = venv/Scripts/python:<class 'str'>


In [ ]:
import numpy as np

a = [[[1, 2], [3,4]],[[5, 6], [7, 8]]]
b = np.array(a)
np.save('test1.npy', b)

In [ ]:
import os
path = r'c:\Users\fujikawa\python_projects\ARIM_testcode\out_l.npy'
splited = os.path.splitext(path)
print(splited)

# queue+importをJupyter上で試す
- 動作しない

In [ ]:
from multiprocessing import Process, Queue, cpu_count, Array

import time
# from multiprocess_queue_test import save_npy
from glob import glob
import os
import numpy as np
import cv2
from PIL import Image
paths = glob('icopng/*.png')
path_list = Array('i', len(paths))
for i,path in enumerate(paths):
    path_list[i] = path
# top_results = [[0, 0] for _ in range(11)]
top_results = Array('i', range(len(path_list)))

save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成

save_base_path = 'gaussian_filtered_nparrays/'
# processに渡す引数の作成

def save_npy(q, top_results, path_list):
    
    while True:
        item = q.get()
        if item is None:
            break
        # 処理のため少しディレイを入れる
        time.sleep(0.1)
        
        # pngデータをopencvでロード
        im = cv2.imread(path_list[item])
        # わざとCPUバウンドの処理を入れる
        im_calc = Image.open(path).convert('L')
        im_calc = np.array(im)
        im_calc = im_calc.astype(np.float64)
        i = 6
        im_b = im_calc
        for _ in range(i):
            im_a = im_calc+np.var(im_calc)
            im_b = im_calc-np.var(im_b)
            im_calc = np.cov(im_b, im_a)
        x = np.argsort(im_calc.flatten())
        # 処理結果をglobalの結果ファイルに代入する
        top_results[item][0] = item
        top_results[item][1] = len(x)
        #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
        file_name = path_list[item].split('\\')[-1].split('.png')[0]
        save_path = save_base_path + file_name + '_' + path_list[item].split('.')[1]
        #全てのnpyファイルを同じフォルダに格納
        np.save(save_path, im)

        print(f'Finished q[{item}] = {path_list[item]}')
        q.task_done()
    #pngデータをopencvでロード


q = Queue()
    
process_list = []
start_t = time.time()
# processを開始
for _ in range(len(path_list)):
    p = Process(target=save_npy, args=(q, top_results, path_list, ), daemon=True)
    p.start()
    process_list.append(p)

# queueの作成
for item in range(len(path_list)):
    q.put(item)

# queueの終了開始
for _ in range(len(path_list)):
    q.put(None)
    
# Block until all tasks are done.
for p_result in process_list:
    p_result.join()
print(f'top result = {top_results}')
print(time.time()-start_t)
print('All work completed')
    



In [2]:
def wrapper(args):
    func(*args, a=1)

def func(b, c, a=None):
    print(f'a, b, c = {a}, {b}, {c}')

wrapper((2, 3))

a, b, c = 1, 2, 3


- https://ja.pymotw.com/2/multiprocessing/communication.html

In [ ]:
import multiprocessing
import time

class Consumer(multiprocessing.Process):
    
    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        proc_name = self.name
        while True:
            next_task = self.task_queue.get()
            if next_task is None:
                # Poison pill は終了を意味します
                print '%s: Exiting' % proc_name
                self.task_queue.task_done()
                break
            print '%s: %s' % (proc_name, next_task)
            answer = next_task()
            self.task_queue.task_done()
            self.result_queue.put(answer)
        return


class Task(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __call__(self):
        time.sleep(0.1) # この処理に少し時間がかかることを意図しています
        return '%s * %s = %s' % (self.a, self.b, self.a * self.b)
    def __str__(self):
        return '%s * %s' % (self.a, self.b)


if __name__ == '__main__':
    # コミュニケーションキューを作成する
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.Queue()
    
    # consumers 処理を開始する
    num_consumers = multiprocessing.cpu_count() * 2
    print 'Creating %d consumers' % num_consumers
    consumers = [ Consumer(tasks, results)
                  for i in xrange(num_consumers) ]
    for w in consumers:
        w.start()
    
    # ジョブをキューへ入れる
    num_jobs = 10
    for i in xrange(num_jobs):
        tasks.put(Task(i, i))
    
    # 各 consumer へ poison pill を追加する
    for i in xrange(num_consumers):
        tasks.put(None)

    # 全てのタスクの終了を待つ
    tasks.join()
    
    # 結果を表示し始める
    while num_jobs:
        result = results.get()
        print 'Result:', result
        num_jobs -= 1

# pathos　ParallelPool、uimap＋for文

In [ ]:
import os
from multiprocessing import cpu_count 
from pathos.pools import ParallelPool
from tqdm import tqdm 
from glob import glob

from multiprocess_test import wrap_save_npy
top_results = [[0, 0] for _ in range(11)]


# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
pool = ParallelPool(node=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')
# processに渡す引数の作成
job_args = [(path, path.split('.')[1], num) for num,path in enumerate(path_list)]
# print(job_args)
wrap_save_npy(job_args[0])
# uimapによってprocessの順番は非同期で処理される
for result in pool.uimap(wrap_save_npy, job_args):
    print(result)
# print(f' top results = {top_results}')

In [ ]:
import os
from multiprocessing import Pool, cpu_count 
from pathos.pools import ParallelPool
from tqdm import tqdm 
from glob import glob
import dill
import pickle
import multiprocess_test3
from multiprocess_test3 import wrap_save_npy
top_results = [[0, 0] for _ in range(11)]

# wrap_save_npy.namespace_main = globals()
# print(wrap_save_npy.namespace_main)
# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
# p = Pool(processes=cpu_count()-1)
p = ParallelPool(node=cpu_count()-1)

# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')
multiprocess_test3.top_namespace = globals()
print(f'globals = {multiprocess_test3.top_namespace}')
# processに渡す引数の作成
print(f'top = {multiprocess_test3.top_namespace["top_results"]}')
job_args = [(path, path.split('.')[1], num, 'top_results') for num,path in enumerate(path_list)]
# print(job_args)
wrap_save_npy(job_args[0])
# imapによってprocessの順番は任意で処理される

# for result in p.imap_unordered(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
for result in p.map(wrap_save_npy, job_args):
    print(result)
# print(f' top results = {top_results}')

In [ ]:
import test_globals_func
a = 10
b = 20
print(globals())
test_globals_func.add("a", "b", "c", globals())
print(c)

In [ ]:
import os
import numpy as np
import cv2
from multiprocessing import Pool, cpu_count 
from tqdm import tqdm 
from glob import glob
from multiprocess_test import wrap_save_npy

p = Pool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)

def wrap_save_npy(args):
    return save_npy(*args)

def save_npy(path, type_name):

    #pngデータをopencvでロード
    im = cv2.imread(path)
    #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
    file_name = path.split('\\')[-1].split('.png')[0]
    save_path = save_base_path + file_name + '_' + type_name
    #全てのnpyファイルを同じフォルダに格納
    np.save(save_path, im)
    return save_path

path_list = glob('icopng/*.png')
print(path_list)
for path in path_list:
    print(path)
job_args = [(path, path.split('.')[1]) for path in path_list]
# wrap_save_npy([path_list[0],'test'])
#list(tqdm(p.imap(wrap_save_npy, job_args), total=len(path_list)))
# p.imap(wrap_save_npy, job_args)

for result in p.imap_unordered(wrap_save_npy, job_args):
    print(result)


In [ ]:
import os
import numpy as np
import cv2
from multiprocessing import Pool, cpu_count 
from tqdm import tqdm 
from glob import glob

p = Pool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)


def wrap_save_npy(args):
    return save_npy(*args)


def save_npy(path, type_name):

    #pngデータをopencvでロード
    im = cv2.imread(path)
    #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
    file_name = path.split('\\')[-1].split('.png')[0]
    save_path = save_base_path + file_name + '_' + type_name
    #全てのnpyファイルを同じフォルダに格納
    np.save(save_path, im)

path_list = glob('icopng/*.png')
print(path_list)
for path in path_list:
    print(path)
job_args = [(path, path.split('.')[1]) for path in path_list]
# wrap_save_npy([path_list[0],'test'])
#list(tqdm(p.imap(wrap_save_npy, job_args), total=len(path_list)))
p.imap(wrap_save_npy, job_args)

p.close()
p.join()


- multiprocessing無し

In [ ]:
import os
from glob import glob
import numpy as np
from PIL import Image

save_base_path = 'gaussian_filtered_nparrays/'
path_list = glob('icopng/*.png')
print(path_list)
results = [0 for _ in range(len(path_list))]
numbers = [i for i in range(len(path_list))]
map_list = [[i, path] for i, path in enumerate(path_list)]


def save_im(path):
    # global save_base_path
    # global results
    #PILでロード
    im = Image.open(path).convert('L')
    

    print(im.getpixel((20, 20)))
    im = np.array(im)
    im = im.astype(np.float64)
    num = 6
    im_b = im
    for _ in range(num):
        im_a = im+np.var(im)
        im_b = im-np.var(im_b)
        im = np.cov(im_b, im_a)

    x = np.argsort(im.flatten())
    y = np.argsort(x)

    return path

for m in list(map(save_im, path_list)):
    print(m)
print(results)


- mapのテスト

In [ ]:
from glob import glob
path_list = glob('icopng/*.png')
print(path_list)
results = [0 for _ in range(len(path_list))]
numbers = [i for i in range(len(path_list))]
map_list = [[i, path] for i, path in enumerate(path_list)]
print(map_list)

def sep(*args):
    print(args)
    print(f'n,f = {args[0][0]},{args[0][1]}')
    return f'n,f = {args[0][0]},{args[0][1]}'
for a in list(map(sep, map_list)):
    print(a)

In [ ]:
a = [[i, 0] for i in range(5)]
print(a)
print(a[1][0])

In [4]:
import random
import time

def mul(a, b):
    time.sleep(0.5*random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5*random.random())
    return a + b

def marge(a, b):
    mul_r = mul(a, b)
    plus_r = plus(a, b)
    return mul_r/plus_r

print(f'{mul(1, 7)}{type(mul(1, 7))}')
print(f'{plus(1, 7)}{type(plus(1, 7))}')
print(f'{marge(1, 7)}{type(marge(1, 7))}')


7<class 'int'>
8<class 'int'>
0.875<class 'float'>
